## ch7 python requests를 이용한 크롤러 개발


### 1. 필요 라이브러리
이제 본격적으로 크롤러를 개발해보겠습니다. 웹 크롤러를 만들기 위해서는 먼저 http client가 있어야 합니다. 지금까지는 웹 브라우저를 클라이언트로 사용하였는데, python의 reuests라는 라이브러리를 사용하면 쉽게 HTTP 요청을 보낼 수 있습니다. 그리고 응답으로 수신한 HTML 문서를 쉽게 파싱하기 위해서 BeautifulSoup이라는 라이브러리를 이요하겠습니다.

- requests
- BeautifualSoup

In [1]:
!pip install requests
!pip install beautifulsoup4

In [2]:
import requests
from bs4 import BeautifulSoup

### 2. news list backend API에 HTTP 요청 보내기
뉴스 기사 목록을 내려주는 네이버 스포츠 백엔드 API에 2023년 5월 10일 뉴스 기사 목록을 요청을 보낸 뒤, 응답을 json 객체로 바꿔보겠습니다. json은 python에서는 dict 데이터 타입으로 표현됩니다.

In [4]:
url = "https://sports.news.naver.com/kbaseball/news/list?isphoto=N&date=20230510"

In [5]:
resp = requests.get(url)

In [9]:
resp

<Response [200]>

In [10]:
resp_json = resp.json()

In [11]:
type(resp_json)

dict

In [13]:
resp_json["list"][0]

{'oid': '425',
 'aid': '0000140099',
 'officeName': '마니아타임즈',
 'title': '[10일 팀 순위표]롯데, 하룻만에 LG 밀어내고 2위 자리 되찾아…삼성과 두산은 한화와 롯데에 덜미 잡히며 5할 승률 무너져',
 'subContent': None,
 'thumbnail': 'http://imgnews.naver.net/image/thumb154/425/2023/05/10/140099.jpg',
 'datetime': '2023.05.10 23:58',
 'url': None,
 'sectionName': 'KBO리그',
 'type': 'PHOTO',
 'totalCount': 320}

In [14]:
resp_json["totalPages"]

25

### 3. 반목문을 돌면서 모든 페이지의 뉴스 기사 oid, aid 수집하기
totalPages 값을 통해서 2023년 5월 10일 야구 뉴스 기사가 총 25 페이지 있다는 것을 알게되었습니다. 우리가 원하는 것은 특정일에 나온 뉴스 기사들의 oid와 aid들입니다. 한번 for문을 돌면서 모든 페이지를 요청하고, oid와 aid를 수집해보겠습니다. 

전체 진행 상황을 모니터링 하기 위해서 tqdm 라이브러리를 사용해보겠습니다.  

In [15]:
total_pages = resp_json["totalPages"] 

In [23]:
from tqdm import tqdm

def parse_news_list(resp):
    oid_aid_tuple_list = []
    resp_json = resp.json()
    for item in resp_json["list"]:
        oid_aid_tuple_list.append((item["oid"], item["aid"]))
    return oid_aid_tuple_list

total_oid_aid_list = []
for i in tqdm(range(total_pages)):
    article_list_url =  f"https://sports.news.naver.com/kbaseball/news/list?isphoto=N&date=20230510&page={i+1}" 
    resp = requests.get(article_list_url)
    oid_aid_tuple_list = parse_news_list(resp)
    total_oid_aid_list.extend(oid_aid_tuple_list)

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:05<00:00,  4.62it/s]


In [24]:
len(total_oid_aid_list)

499

In [25]:
total_oid_aid_list[:10]

[('425', '0000140099'),
 ('425', '0000140098'),
 ('076', '0004005847'),
 ('422', '0000598002'),
 ('117', '0003727891'),
 ('023', '0003762909'),
 ('421', '0006799173'),
 ('032', '0003222806'),
 ('311', '0001591181'),
 ('109', '0004847557')]

### 4. 뉴스 기사 페이지를 요청한 뒤, 기사 제목과 본문 파싱하기 
기사 상세 페이지를 하나만 요청한 뒤, BeautifulSoup을 이용해 파싱해보겠습니다. BeautifulSoup 객체를 만들 때 뒤에 붙여주는 "lxml"은 BeautifulSoup에 내장된 HTML 파서 중 가장 많이 사용되는 것입니다.

In [65]:
article_url = "https://sports.news.naver.com/news?oid=139&aid=0002183715"

In [66]:
resp = requests.get(article_url)

In [67]:
soup = BeautifulSoup(resp.text, "lxml")

In [68]:
# 제목 파싱
title_text = soup.find("h4", class_="title").get_text()

In [69]:
title_text

"'평균 구속 143.2km/h' 돌직구가 사라진 최고령 투수 오승환이 살아남는 법"

In [70]:
# 본문 파싱
newsEndContents = soup.find("div", id="newsEndContents")

In [72]:
newsEndContents

<div class="news_end" id="newsEndContents">
<span class="end_photo_org"><img alt="" src="https://imgnews.pstatic.net/image/139/2023/05/22/0002183715_001_20230522225401253.jpg?type=w647"/></span> <br/> [스포탈코리아] 김경현 기자= 삼성 라이온즈의 끝판왕 오승환이 생존을 위해 끊임없이 발전하고 있다.<br/>잡<br/>오승환은 21일 창원 NC 다이노스전 1-1 동점 상황 연장 11회 말 등판했다. 선두타자 마틴에게 볼넷을 내줬지만 권희동을 5-4-3 병살타로 잡아내며 한숨을 돌린 오승환. 후속타자 박세혁에게 5연속 직구를 던지며 2루수 땅볼로 이닝을 마쳤다.<br/><br/>12회 초 공민규의 3루타와 강민호의 희생플라이로 역전에 성공한 삼성. 오승환은 세이브 상황은 아니지만 삼성의 승리를 지키기 위해 12회 말에도 마운드에 올랐다. 선두타자 김한별을 3루 땅볼로 잡아냈지만 도태훈에게 볼넷, 김주원에게 몸에 맞은 공이 나오며 1사 1, 2루 위기에 몰렸다. 오승환은 돌부처답게 흔들리지 않았다. 천재환을 중견수 뜬공, 서호철을 3루 땅볼로 처리하며 경기를 마무리했다.<br/><br/>이날 오승환은 KBO 통산 39번째 승리를 거두었다. 2이닝 동안 총 32구를 던졌으며 스트라이크 비율은 56.3%에 달했다.<br/><br/>21일 오승환은 커브의 구사율을 15.6%까지 올렸다. 2023 시즌 중 2번째로 높은 수치다.(4월 13일 20.8%) 오승환의 올 시즌 커브 구사 비율은 8.7%로 한국 복귀 이후 가장 높다.(2020년 4.6%, 2021년 6.0%, 2022년 4.4%)<br/><br/>오승환의 트레이드 마크는 시속 150km를 넘나드는 강속구였다. 하지만 지금 오승환은 82년생으로 리그 최고령 투수가 됐고 이전과 같은 강속구를 찾아볼 수 없다. 이번 시즌 오승환의 패스트볼 평균 구속은 시속 14

기사 본문 외에도 기자 명이나 언론사 링크 등의 불필요한 정보들이 포함되어 있다. 예를 들어 div id="newsEndContents" 태그 안에 포함된 p, div, span, em 태그들은 모두 불필요한 텍스트들을 가지고 있다. 이를 BeautifulSoup를 이용해서 제거한다. 

In [76]:
def _remove_tags(parent_soup, target_tag):
    tags = parent_soup.find_all(target_tag)
    for tag in tags:
        tag.decompose()

In [80]:
_remove_tags(newsEndContents, "p")
_remove_tags(newsEndContents, "div")
_remove_tags(newsEndContents, "em")
_remove_tags(newsEndContents, "span")
article = newsEndContents.get_text(separator=" ").strip()
article = article.replace("\xa0", " ")

In [81]:
article

'[스포탈코리아] 김경현 기자= 삼성 라이온즈의 끝판왕 오승환이 생존을 위해 끊임없이 발전하고 있다. 잡 오승환은 21일 창원 NC 다이노스전 1-1 동점 상황 연장 11회 말 등판했다. 선두타자 마틴에게 볼넷을 내줬지만 권희동을 5-4-3 병살타로 잡아내며 한숨을 돌린 오승환. 후속타자 박세혁에게 5연속 직구를 던지며 2루수 땅볼로 이닝을 마쳤다. 12회 초 공민규의 3루타와 강민호의 희생플라이로 역전에 성공한 삼성. 오승환은 세이브 상황은 아니지만 삼성의 승리를 지키기 위해 12회 말에도 마운드에 올랐다. 선두타자 김한별을 3루 땅볼로 잡아냈지만 도태훈에게 볼넷, 김주원에게 몸에 맞은 공이 나오며 1사 1, 2루 위기에 몰렸다. 오승환은 돌부처답게 흔들리지 않았다. 천재환을 중견수 뜬공, 서호철을 3루 땅볼로 처리하며 경기를 마무리했다. 이날 오승환은 KBO 통산 39번째 승리를 거두었다. 2이닝 동안 총 32구를 던졌으며 스트라이크 비율은 56.3%에 달했다. 21일 오승환은 커브의 구사율을 15.6%까지 올렸다. 2023 시즌 중 2번째로 높은 수치다.(4월 13일 20.8%) 오승환의 올 시즌 커브 구사 비율은 8.7%로 한국 복귀 이후 가장 높다.(2020년 4.6%, 2021년 6.0%, 2022년 4.4%) 오승환의 트레이드 마크는 시속 150km를 넘나드는 강속구였다. 하지만 지금 오승환은 82년생으로 리그 최고령 투수가 됐고 이전과 같은 강속구를 찾아볼 수 없다. 이번 시즌 오승환의 패스트볼 평균 구속은 시속 143.2km로 한국 복귀 이후 가장 낮다. 2023년 KBO리그 패스트볼 평균 구속은 시속 143.5km로, 오승환은 커리어 최초로 리그 평균보다 느린 패스트볼을 던지고 있다. 패스트볼 구속이 떨어지자 위력 역시 자연스럽게 내려갔다. 오승환은 2020년 한국에 복귀하고 평균 시속 146.2km의 패스트볼을 던졌으며 구종 가치는 4.8을 기록했다. 2021년 패스트볼 평균 구속은 시속 145.7km였으며 구종 가치 9.6로 정점을 찍었다

### 5. 코드 정리 및 CSV 파일에 데이터 쓰기
지금까지 네이버 스포츠에서 특정 요일에 야구 기사 목록을 가져온 뒤, 각각의 기사의 제목과 본문을 수집하는 크롤러를 개발해보았다. 코드를 정리해보면 아래와 같다.

In [83]:
target_date = "20230510"
article_list_url_format = "https://sports.news.naver.com/kbaseball/news/list?isphoto=N&date={date}&page={page}"

In [95]:
def get_total_pages(date):
    """
    전체 페이지 수를 알기 위해 첫 페이지 요청을 보낸 뒤, 전체 페이지 수만 리턴
    """
    resp = requests.get(article_list_url_format.format(date=date, page=1))
    resp_json = resp.json()
    total_pages = resp_json["totalPages"]
    return total_pages

In [97]:
total_pages = get_total_pages(target_date)
print("total_pages:", total_pages)

total_pages: 25


In [111]:
#  뉴스 기사 목록 파싱 로직을 함수화 한 뒤, 반복문을 돌며 요청을 보내어 oid, aid 수집
from tqdm import tqdm

def parse_news_list(resp):
    """
    스포츠 뉴스 목록 응답을 파싱하는 함수
    각 아이템별로 oid와 aid를 추출한 뒤, 튜플로 묶어주어 list에 담은 뒤 리턴
    """
    oid_aid_tuple_list = []
    resp_json = resp.json()
    for item in resp_json["list"]:
        oid_aid_tuple_list.append((item["oid"], item["aid"]))
    return oid_aid_tuple_list

def crawl_article_oid_aid(date, total_pages):
    """
    반복문을 돌며 각 뉴스 기사 페이지 요청 및 파싱
    """
    total_oid_aid_list = []
    total_pages = get_total_pages(date) 
    for i in tqdm(range(total_pages)):
        article_list_url = article_list_url_format.format(date=date, page=(i+1))
        resp = requests.get(article_list_url)
        oid_aid_tuple_list = parse_news_list(resp)
        total_oid_aid_list.extend(oid_aid_tuple_list)
    return total_oid_aid_list

In [112]:
total_oid_aid_list = crawl_article_oid_aid(target_date, total_pages)

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:04<00:00,  5.15it/s]


In [106]:
print("total_oid_aid_list", len(total_oid_aid_list))

total_oid_aid_list 499


In [107]:
def remove_tags(parent_soup, target_tag):
    """
    BeautifulSoup 객체에서 불필요한 태그를 삭제하는 함수
    """
    tags = parent_soup.find_all(target_tag)
    for tag in tags:
        tag.decompose()

In [2]:
def parse_article(resp):
    soup = BeautifulSoup(resp.text, "lxml")
    # 제목 파싱
    title = soup.find("h4", class_="title").get_text()
    
    # 본문 파싱
    content_soup = soup.find("div", id="newsEndContents")
    _remove_tags(content_soup, "p")
    _remove_tags(content_soup, "div")
    _remove_tags(content_soup, "em")
    _remove_tags(content_soup, "span")
    content = content_soup.get_text().strip()
    return title, content

In [1]:
import csv

article_url_format = "https://sports.news.naver.com/news?oid={oid}&aid={aid}"

def crawl_articles(oid_aid_list):
    """
    특정 기사의 oid와 aid를 담은 list를 받아서 각 페이지를 요청한 뒤,
    기사의 제목과 본문을 파싱하여 csv 파일에 쓰는 함수
    """
    with open("./baseball_news.csv", "w") as fw:
        writer = csv.writer(fw)
        writer.writerow([""])
        for oid, aid in tqdm(oid_aid_list, total=len(oid_aid_list)):
            article_url = article_url_format.format(oid=oid, aid=aid)
            resp = requests.get(article_url)
            # 파싱 에러 처리
            try:
                title, content = parse_article(resp)
                if not title or not content:
                    continue
            except Exception as e:
                print(e, article_url)
                continue
            writer.writerow([article_url, title, content])

In [122]:
crawl_articles(total_oid_aid_list)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 499/499 [02:29<00:00,  3.33it/s]
